# AI CA1 Genetics 

# Ali Darabi  -  810100264

# Part 1 : Define Basic Concepts : 

First, we need to define our genes and chromosomes. The genes will be coefficients, so each gene represents one of the coefficients. If the input degree of the polynomial is n,each chromosome will contain (n + 1) genes. This means each chromosome is a potential solution to our problem. For example, if we input degree n = 3, we have 4 coefficients, so each chromosome contains 4 genes.

Next, we define parameters to solve our problem. Some of these parameters set the crossover method we use to solve problems, such as uniform or single-point crossover. We define two data classes, one to hold hyperparameters (HyperParameters) and the other to hold our problem inputs (ProblemInputs).

In [1]:
import random
from dataclasses import dataclass

SINGLE_POINT_CROSSOVER = "SINGLE POINT"
UNIFORM_CROSSOVER = "UNIFORM"

@dataclass
class HyperParameters:
    CROSSOVER_PROBABILITY: float
    METHOD_OF_CROSSOVER: str
    MUTATION_PROBABILITY: float
    NUM_OF_CHROMOSOMES: int
    CARRY_TO_NEXT_ROUND_RATIO: float
    NUMBER_OF_GENERATIONS: int

@dataclass
class ProblemInputs:
    degree: int
    range_start: int
    range_end: int 
    points: list[list[int]]

# Part 2 : Define Primary Population :

This function creates a primary population of chromosomes, using randint to create random genes. The inputs are the number of chromosomes, the number of genes per chromosome, the range start, and the range end.

In [2]:
def primary_population_production(NUM_OF_CHROMOSOMES, num_of_genes, range_start, range_end):
    population = []
    for i in range(NUM_OF_CHROMOSOMES):
        random_genes = [(random.randint(range_start, range_end)) for x in range(num_of_genes)]
        population.append(random_genes)
    return population

# Part 3 : Implementation and Fitness function :

This function takes a chromosome (coefficients) and an x-value and calculates the corresponding y-value.

In [3]:
def calc_polynomial(coefficients, x):
    calculated_y = 0
    degree = len(coefficients) - 1
    for i in range(len(coefficients)):
        calculated_y += coefficients[i] * (x ** (degree - i))
    return calculated_y

In the fitness function, we calculate the sum of the absolute errors between the calculated y-values and the actual y-values for a chromosome and all given points. The fitness value is then calculated as the inverse of the error sum plus one. We add one to avoid dividing by zero in the case of a perfect solution.

Errors = | actual y_values - calculated y_values |


Fitness(Errors) = 1 / (Errors + 1)

In [4]:
def fitness_func(chromosome, points):
    errors_sum = 0
    for x, y in points:
        calculated_y = calc_polynomial(chromosome, x)
        errors_sum += abs((y - calculated_y))
    return ((1 / (errors_sum + 1)) * 100)

We use this function to calculate the fitness value for all chromosomes.

In [5]:
def calc_chromosomes_fitness(chromosomes, points):
    chromosomes_fitness = []
    for i in range(len(chromosomes)):
        chromosomes_fitness.append(fitness_func(chromosomes[i], points))
    return chromosomes_fitness

This function works by first calculating the sum of the fitness values for all chromosomes.Then, it calculates the probabilities of each chromosome being selected for the mating pool by dividing the fitness of each chromosome by the sum of the fitness values for all chromosomes.
Finally, the function builds the mating pool of chromosomes by randomly selecting chromosomes with probabilities proportional to their fitness values.This means that chromosomes with higher fitness values have a higher chance of being selected for the mating pool.

In [6]:
def build_mating_pool(chromosomes, chromosomes_fitness):
    sum_fitness = sum(chromosomes_fitness)
    probabilities_weights = []
    for fitness in chromosomes_fitness:
        probabilities_weights.append(fitness / sum_fitness)
    crossover_pool = []
    for j in range(len(chromosomes)):
        selected_chromosome = random.choices(range(len(chromosomes)), weights=probabilities_weights)[0]
        crossover_pool.append(chromosomes[selected_chromosome])
    return crossover_pool

# Part 4 : Crossover and Mutation :

This function takes two chromosomes as input and returns two new offspring chromosomes. The crossover method is determined by the const variable METHOD_OF_CROSSOVER.
If the crossover method is SINGLE_POINT_CROSSOVER, a single crossover point is randomly selected and the two chromosomes are exchanged at that point. If the crossover method is UNIFORM_CROSSOVER, each gene in the offspring chromosomes is randomly selected from either of the parent chromosomes with equal probability.

In [7]:
def crossover_func(chromosome_one, chromosome_two, CROSSOVER_PROBABILITY, METHOD_OF_CROSSOVER):
    if METHOD_OF_CROSSOVER == SINGLE_POINT_CROSSOVER:
        if random.random() < CROSSOVER_PROBABILITY:
            crossover_point = random.randint(1, len(chromosome_one) - 1)
            new_chromosome_one = chromosome_one[:crossover_point] + chromosome_two[crossover_point:]
            new_chromosome_two = chromosome_two[:crossover_point] + chromosome_one[crossover_point:]
            return new_chromosome_one, new_chromosome_two
        else:
            return chromosome_one, chromosome_two
    elif METHOD_OF_CROSSOVER == UNIFORM_CROSSOVER:
        new_chromosome_one = []
        new_chromosome_two = []
        for i in range(len(chromosome_one)):
            if random.random() < CROSSOVER_PROBABILITY:
                new_chromosome_one.append(chromosome_two[i])
                new_chromosome_two.append(chromosome_one[i])
            else:
                new_chromosome_one.append(chromosome_one[i])
                new_chromosome_two.append(chromosome_two[i])
        return new_chromosome_one, new_chromosome_two

This function works by first sorting the chromosomes by their fitness values, in descending order. Then, it selects the top chromosomes as the elite chromosomes.

In [8]:
def elite_chromosomes_selection(chromosomes, chromosomes_fitness, CARRY_TO_NEXT_ROUND_RATIO):
    chromosomes_sorted = sorted(zip(chromosomes, chromosomes_fitness), key=lambda x: x[1], reverse=True)
    elite_chromosomes = [x for x, _ in chromosomes_sorted[:int(CARRY_TO_NEXT_ROUND_RATIO * len(chromosomes))]]
    return elite_chromosomes

This function works by first creating a copy of the chromosomes list. Then, it removes the elite chromosomes from the ordinary chromosomes list.

In [9]:
def ordinery_chromosomes_selection(chromosomes, elite_chromosomes):
    ordinery_chromosomes = chromosomes
    for i in range(len(elite_chromosomes)):
        for j in range(len(ordinery_chromosomes)):
            if elite_chromosomes[i] == ordinery_chromosomes[j]:
                ordinery_chromosomes.remove(ordinery_chromosomes[j])
                break
    return ordinery_chromosomes

This function works by first selecting the elite chromosomes and the ordinary chromosomes. The elite chromosomes are the chromosomes with the highest fitness values in the population, and they are not mutated. The ordinary chromosomes are the remaining chromosomes in the population, and they are mutated.Once the ordinary chromosomes have been mutated, the function combines them with the elite chromosomes to form the new population of chromosomes.

In [10]:
def mutation_func(chromosomes, chromosomes_fitness, range_start, range_end, MUTATION_PROBABILITY, CARRY_TO_NEXT_ROUND_RATIO):
    elite_chromosomes = elite_chromosomes_selection(chromosomes, chromosomes_fitness, CARRY_TO_NEXT_ROUND_RATIO)
    ordinery_chromosomes = ordinery_chromosomes_selection(chromosomes, elite_chromosomes)
    for chromosome in ordinery_chromosomes:
        for i in range(len(chromosome)):
            if random.random() < MUTATION_PROBABILITY:
                chromosome[i] += round(random.uniform(range_start, range_end))
                if chromosome[i] > range_end:
                    chromosome[i] = range_end
                elif chromosome[i] < range_start:
                    chromosome[i] = range_start

    chromosomes_mutated = elite_chromosomes + ordinery_chromosomes
    return chromosomes_mutated

# Part 5 : Implementing the Genetic Algorithm on problem 

Finally, we use the functions we declared in other parts to apply the genetic algorithm to our problem.

This function applies crossover to a population of chromosomes and returns a new population of chromosomes.

In [11]:
def apply_crossover(chromosomes, chromosomes_fitness, CROSSOVER_PROBABILITY, METHOD_OF_CROSSOVER):
    crossover_pool = build_mating_pool(chromosomes, chromosomes_fitness)
    new_chromosomes = []
    for i in range(0, len(crossover_pool), 2):
        parent_one = crossover_pool[i]
        parent_two = crossover_pool[i + 1]
        child_one, child_two = crossover_func(parent_one, parent_two, CROSSOVER_PROBABILITY, METHOD_OF_CROSSOVER)
        new_chromosomes.append(child_one)
        new_chromosomes.append(child_two)
    return new_chromosomes

This function prints the best chromosome and its fitness value.

In [12]:
def print_answer(chromosomes, chromosomes_fitness):
    best_chromosome = chromosomes[0]
    best_fitness = chromosomes_fitness[0]
    for i in range(1, len(chromosomes)):
        if chromosomes_fitness[i] > best_fitness:
            best_chromosome = chromosomes[i]
            best_fitness = chromosomes_fitness[i]

    for i in range(len(best_chromosome)):
        print("coefficient of x ^",(len(best_chromosome) - i - 1), "is:", best_chromosome[i])
    print("Fitness:", best_fitness)


This function applies genetic algorithms to th problem to find the best solution and we put a limit on number of generations.

In [13]:
def apply_genetics_on_problem(problem_inputs: ProblemInputs, hyper_parameters: HyperParameters):
    num_of_genes = problem_inputs.degree + 1
    range_start = problem_inputs.range_start
    range_end = problem_inputs.range_end
    points = problem_inputs.points
    NUM_OF_CHROMOSOMES = hyper_parameters.NUM_OF_CHROMOSOMES
    NUMBER_OF_GENERATIONS = hyper_parameters.NUMBER_OF_GENERATIONS
    CROSSOVER_PROBABILITY = hyper_parameters.CROSSOVER_PROBABILITY
    METHOD_OF_CROSSOVER = hyper_parameters.METHOD_OF_CROSSOVER
    MUTATION_PROBABILITY = hyper_parameters.MUTATION_PROBABILITY
    CARRY_TO_NEXT_ROUND_RATIO = hyper_parameters.CARRY_TO_NEXT_ROUND_RATIO

    chromosomes = primary_population_production(NUM_OF_CHROMOSOMES, num_of_genes, range_start, range_end)
    for generation in range(NUMBER_OF_GENERATIONS):
        chromosomes_fitness = calc_chromosomes_fitness(chromosomes, points)
        chromosomes = apply_crossover(chromosomes, chromosomes_fitness, CROSSOVER_PROBABILITY, METHOD_OF_CROSSOVER)
        chromosomes_fitness = calc_chromosomes_fitness(chromosomes, points)
        chromosomes = mutation_func(chromosomes, chromosomes_fitness, range_start, range_end, MUTATION_PROBABILITY, CARRY_TO_NEXT_ROUND_RATIO)
    print_answer(chromosomes, chromosomes_fitness)

For testing our algorithm, we use the following parameters:

In [14]:
hyper_parameters = HyperParameters(
    CROSSOVER_PROBABILITY = 0.6,
    METHOD_OF_CROSSOVER = SINGLE_POINT_CROSSOVER, 
    MUTATION_PROBABILITY = 0.2,
    NUM_OF_CHROMOSOMES = 500,
    CARRY_TO_NEXT_ROUND_RATIO = 0.1,
    NUMBER_OF_GENERATIONS = 300
) 

Initializing our problem inputs :

In [38]:
problem_inputs = ProblemInputs(
    degree = 2,
    range_start = -10,
    range_end = 10,
    points = [[0, 1], [-1, 0], [1, 5]]
)

In [39]:
apply_genetics_on_problem(problem_inputs, hyper_parameters)

coefficient of x ^ 2 is: 1
coefficient of x ^ 1 is: 3
coefficient of x ^ 0 is: 1
Fitness: 50.0


The final answer depends on the range of coefficients we choose, but one optimal answer is [1, -5, 3, 1].

I will write the answers for Part 6 in a PDF file, which I will send to you along with this file.